<a href="https://colab.research.google.com/github/SHRISHTI1414/AIRL-CV-Intern-Assignment/blob/main/q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
from PIL import Image

uploaded = files.upload()

for filename in uploaded.keys():
    image = Image.open(filename).convert("RGB")
    print(f"Loaded image: {filename}")


Saving dog.jpeg to dog.jpeg
Loaded image: dog.jpeg


In [ ]:
# INSTALL DEPENDENCIES

!pip install git+https://github.com/IDEA-Research/GroundingDINO.git
!pip install git+https://github.com/facebookresearch/segment-anything-2.git
!pip install opencv-python matplotlib torch torchvision


# IMPORTS

import torch
import cv2
import matplotlib.pyplot as plt
from groundingdino.util.inference import Model
from segment_anything import sam_model_registry, SamPredictor

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# LOAD MODELS

# GroundingDINO model
config_path = "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.cfg.py"
checkpoint_path = "groundingdino_swint_ogc.pth"
grounding_model = Model(model_config_path=config_path, model_checkpoint_path=checkpoint_path)
grounding_model = grounding_model.to(DEVICE)

# SAM 2 model
sam_checkpoint = "sam2_hiera_tiny.pt"
sam_model = sam_model_registry["vit_t"](checkpoint=sam_checkpoint)
sam_predictor = SamPredictor(sam_model)
sam_predictor.model = sam_predictor.model.to(DEVICE)


# LOAD YOUR IMAGE

image_path = "dog.jpeg"
image_bgr = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(6,6))
plt.imshow(image_rgb)
plt.title("Original Image")
plt.axis("off")
plt.show()




text_prompt = "dog"

# TEXT-TO-REGION (GroundingDINO)

boxes, logits, phrases = grounding_model.predict_with_caption(
    image_rgb, caption=text_prompt, box_threshold=0.35, text_threshold=0.25
)

print("Detected regions for:", text_prompt)
for p in phrases:
    print("→", p)

# Draw bounding boxes
for box in boxes:
    x0, y0, x1, y1 = [int(v) for v in box]
    cv2.rectangle(image_rgb, (x0, y0), (x1, y1), (255,0,0), 2)

plt.figure(figsize=(6,6))
plt.imshow(image_rgb)
plt.title(f"Detected regions for '{text_prompt}'")
plt.axis("off")
plt.show()


# SEGMENT WITH SAM 2

input_box = boxes[0]  # take first detected region
sam_predictor.set_image(image_rgb)
masks, _, _ = sam_predictor.predict(box=input_box, multimask_output=False)

# Overlay mask
mask = masks[0]
image_masked = image_rgb.copy()
image_masked[~mask] = 0

plt.figure(figsize=(8,8))
plt.imshow(image_rgb)
plt.imshow(mask, alpha=0.5)
plt.title(f"Segmentation for '{text_prompt}'")
plt.axis("off")
plt.show()


  Cloning https://github.com/IDEA-Research/GroundingDINO.git to /tmp/pip-req-build-m7kfazt2
  Running command git clone --filter=blob:none --quiet https://github.com/IDEA-Research/GroundingDINO.git /tmp/pip-req-build-m7kfazt2
  Resolved https://github.com/IDEA-Research/GroundingDINO.git to commit 856dde20aee659246248e20734ef9ba5214f5e44
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 16.2 MB/s eta 0:00:00
  Created wheel for groundingdino: filename=groundingdino-0.1.0-py3-none-any.whl size=87877 sha256=8fd89a4b2547a68f268ba86c748a218f3e467cf2afc5096a78146bed9ef2f71b
  Stored in directory: /tmp/pip-ephem-wheel-cache-dvbhfs40/wheels/3e/eb/2f/04ea637616a335c1f4d624463f273a39f93e819077a8e8d1a2
Successfully built groundingdino
